In [1]:
import pickle 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import nltk
from gensim.models import KeyedVectors
nltk.download('punkt')
 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv("data/capitals.txt", delimiter = ' ')
data.columns = ['city1', 'country1', 'city2' , 'country2']
# print first five elements in the department 
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [3]:
data.shape

(4951, 4)

In [4]:
embeddings = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
f = open('data/capitals.txt','r').read()
set_words = set(nltk.word_tokenize(f))
select_words = words = ['King', 'queen', 'oil', 'gas', 'happy', 'sad' , 'city' , 'town', 'village', 'country', 'continient', 'petroleum', 'joyful']

for w in select_words:
    set_words.add(w)



In [5]:
def get_word_embeddings(embeddings):
    word_embeddings = {}
    for word in embeddings.key_to_index:
        if word in set_words:
            word_embeddings[word] = embeddings[word]
    return word_embeddings
    
# testing the function 
word_embeddings = get_word_embeddings(embeddings)
print(len(word_embeddings))


242


In [6]:
pickle.dump( word_embeddings, open( "word_embeddings_subset.p", "wb" ) )

Now we will load the word embeddings as a python dictionary. As stated these have already been obtained through a machine learning algorithm. 

In [7]:
word_embeddings = pickle.load(open("data/word_embeddings_subset.p", "rb"))
len(word_embeddings)

243

Each of the word embedding is a 300 dimensional vector. 

In [8]:
print("dimension: {}".format(word_embeddings['Spain'].shape[0]))

dimension: 300


Now you will write a function that will use the word embeddings topredict relationships among words. 

The input will take as input three words.
The first two are related to each other. 
It will predict a 4th word which is related to third word in a similar manner as the two first two words are related to each other. 
As an example,"Athens is to Greece as Bangkok is to __"?
You wiull write a program that is capable of finding the fourth word. 
We will give you a hint to show how to compute this. 

A smimilar analogy would be the following:
IN chess:
king - man + woman = queen 


The cosine Similarity function is:
cos(theta)A.B/||A||||B|| = sumation of (prodcut) of AB)/ root of sqaure A product squatre  root of B 

A and B represent the word vectors and A or B represent index I of the vector. Note that if A and B are identical, you will get cos(theta) = 1. 

otherwise if they are the opposite, meainng, A = -B, then you would get cos(theta) = -1
if you get cos(theta) = 0 , that means that they are orthogonal (or perpendicular). 
Numbers between 0 and 1 indicates a similarity score. 
Numbers between -1 and 0 indicates a dissimilarity score. 






# Excercise 1 Cosine_Similarity

implement a function that takes in two word vectors and computes the cosine distance. 


In [9]:
def cosine_similarity(A,B):
    '''Input:
            A: a numpy array which corresponds to a word vector
            B: A numpy array which correcsponds to a word vector
        Output:
            cos: numerical number representing the cosine similariy between A and . 
    '''

    ### Start COde Here ### 
    dot = np.dot(A,B)
    norma = np.linalg.norm(A)  
    normb = np.linalg.norm(B)
    cos = dot/(norma*normb)

    return cos

In [10]:
king = word_embeddings['king']
queen = word_embeddings['queen']

cosine_similarity(king, queen)

0.6510956

# Euclidean Distance 
You will now implkement a function that computes the suiimilarity between two vectors using the Euclidean distanc. Euclidean distance is defined as :

d(A,B) = d(B,A) = root of square of (A-B)^2 + (A-B)^2...+(A - B) ^ 2

n i sthe number of elements in the vector 
A and B are the corresponding word vectors. 
The more similar the words, the more likley the Euclidean distance will be close to 0. 

# Euclidean
Implement a function that computesthe Euclidean distance between two vectors. 



In [11]:
def euclidean(A, B):
    '''' INput:
            A: a numpy array which corresponds to a word vector
            B: a numpy aray which correspojnse to a wprd vector
        Output:
        d: numerical number representing the Euclidean distance between A and B. 
    '''

    d = np.sqrt(np.sum((A-B)**2))

    return d

In [12]:
# Test your function
euclidean(king, queen)

2.4796925

# 1.4 FInding the country of each Capital 
Now, you will use the previous  functions to compute similarities between vectors, and use these to find the capital cities of countries. You will write a function that takes in three word, and the embeding dictionary. Your task is to find the capital citiesr. FOr examples, given the following words:

1 Athens 2: Grece 3: Baghdad, 

Your taks is to predict the country4: Iraq.

# Exercise 3 - get_country

### Instructions:
1. TO predict the capital you might want to look at the King - Man + Woman = Queen exampleabove, and implement that scheme into a mathematical functiona, using the word embeddings and a similarity function. 

2. Iterate over the embedings dictionary and compute the cosine similarity score between your vector abd the current word embedding. 
3. You should add a check to make sure that the word you return is not any of the words that you fed in your function. Return the one with the highest score. 


In [14]:
def get_country(city1, country1, city2, embeddings, cosine_similarity = cosine_similarity):
    """

    """
    # storing the city1 country1 and city2 in a set called group
    group = [city1, country1, city2]

    # get embeddings of the city1
    city1_emb = embeddings[city1]

    # get_embedding of country 1
    country1_emb = embeddings[country1]

    #get embedding of city2
    city2_emb = embeddings[city2]

    # get embedding of country2 
    vec = (country1_emb - city1_emb) + city2_emb

    # Intialize the similarity to -1 (it will be replaced by  a smilarities that aree closer to +1)
    similarity = -1

    # intialize country to empty string
    country = ''

    # Loop through all words in the embeddings dictionary
    for word in embeddings.keys():
        #first check if the word is in group
        if word not in group:
             # get the word embedding
             word_emb = embeddings[word]

             # calculate the cosine similarity 
             cur_similarity = cosine_similarity( vec , word_emb)

             if cur_similarity> similarity:
                 similarity = cur_similarity
                 country = (word, similarity)
    return country



In [15]:
get_country('Athens', 'Greece', 'Cairo', word_embeddings)

('Egypt', 0.7626821)

# Model Accuracy 

Checking the model accuracy of the new function on the dataset

Accuracy = correct # of predictions/ Toral # of predictions

In [16]:
def get_accuracy(word_embeddings, data, get_country = get_country):
    ''''''
    #initialize num correct to zero
    num_correct = 0
    # Loop through the rows of the dataframe
    for i , row in data.iterrows():
        city1 = row['city1']

        country1 = row['country1']

        city2 = row['city2']

        country2 = row['country2']

        predcited_country2,_ = get_country(city1,country1, city2, word_embeddings )

        if predcited_country2 == country2:
            # increment the number of correct by 1 
            num_correct +=1

    m = len(data)
    
    #calculate the accuracy by dividing the number correct by m
    accuracy = num_correct/m

    return accuracy

In [17]:
accuracy = get_accuracy(word_embeddings, data)
print(f"Accuracy is {accuracy:.2f}")

Accuracy is 0.92


# Compute PCA

In [18]:
def get_vectors(words, embedding):
    X = np.zeros((1,300))
    for word in words:
        if word in word_embeddings:
            x = embedding[word]
            X = np.row_stack((X,x))
    X = X[1:,:]
    return X



In [19]:
vec = get_vectors(select_words,word_embeddings)
type(vec) , 

(numpy.ndarray,)

In [34]:
def compute_pca(X,n_components = 2):
    """
    """
    # Mean Centerd the data
    X_mean = np.mean(X,axis=0)
    centered_data  = X - X_mean

    # Calculate Covariance matrix
    cov_matrix = np.cov(centered_data.T)

    # print(len(cov_matrix))
    


    eigen_vals, eigen_vecs = np.linalg.eig(cov_matrix)

    # Sort eigenvalue in increasing order (get the indices from the sort)
    idx_sorted = np.argsort(eigen_vals)

    print(idx_sorted)
    print("---------------------")

    print(eigen_vals)
    print("---------------------")
    print(eigen_vecs)
    # reverse the order so that it's from highest to lowest.
    idx_sorted_decreasing = idx_sorted[::-1]
    print("---------------------")
    print(idx_sorted_decreasing)
    print("---------------------")

    # sort the eigen values by idx_sorted_decreasing
    eigen_vals_sorted = eigen_vals[idx_sorted_decreasing]

    # sort eigenvectors using the idx_sorted_decreasing indices
    eigen_vecs_sorted = eigen_vecs[idx_sorted_decreasing]

    # select the first n eigenvectors 
    # (n is desired dimension of rescaled data array, or n_components)

    eigen_vecs_subset = eigen_vecs_sorted[:,:n_components]

    X_reduced = np.dot(eigen_vecs_subset.T,centered_data.T)

    X_reduced = X_reduced.T



    return X_reduced

    

Testing function

In [35]:
np.random.seed(1)
X = np.random.rand(3,10)
X_reduced = compute_pca(X, n_components = 2)
print("Your Original Matrix was "+ str(X.shape) + " and it became: ")
print(X_reduced)

[0 3 5 4 8 9 7 6 2 1]
---------------------
[-5.55111512e-17+0.00000000e+00j  5.48501886e-01+0.00000000e+00j
  2.50881048e-01+0.00000000e+00j -1.78861952e-17+0.00000000e+00j
 -1.05623286e-17+7.71792418e-18j -1.05623286e-17-7.71792418e-18j
  7.81747807e-18+6.45286014e-18j  7.81747807e-18-6.45286014e-18j
 -2.81696942e-18+0.00000000e+00j  5.20502801e-19+0.00000000e+00j]
---------------------
[[-9.54473629e-01+0.j          2.98289378e-01+0.j
  -1.88121436e-03+0.j          6.20046919e-03+0.j
  -4.39195956e-02-0.07436004j -4.39195956e-02+0.07436004j
  -1.31785887e-02-0.06563523j -1.31785887e-02+0.06563523j
  -1.74129395e-02+0.j          6.08961426e-02+0.j        ]
 [ 6.46841109e-02+0.j          2.06739077e-01+0.j
  -3.78517564e-02+0.j         -1.13742091e-01+0.j
   1.06076211e-01-0.04403154j  1.06076211e-01+0.04403154j
  -1.48964985e-01-0.32115878j -1.48964985e-01+0.32115878j
  -5.02480557e-02+0.j         -1.76582172e-02+0.j        ]
 [ 5.14355546e-02+0.j          1.65857022e-01+0.j
   2.017

In [ ]:
words = ['oil', 'gas', 'happy', 'sad', 'city', 'town',
         'village', 'country', 'continent', 'petroleum', 'joyful']

In [ ]:
X = get_vectors(words,word_embeddings)

In [ ]:
result.shape

In [ ]:
result = compute_pca(X,2)
plt.scatter(result[:,0], result[:,1])
for i , word in enumerate(words):
    plt.annotate(word, xy= (result[i,0] - 0.05, result[i,1]+0.1))

plt .show()

In [36]:
import numpy as np

def compute_pca(X, n_components=2):
    """
    Perform Principal Component Analysis (PCA) on the dataset X and reduce 
    its dimensionality to n_components.
    
    Parameters:
    X : numpy.ndarray
        The input data matrix, where rows represent samples and columns represent features.
    n_components : int
        The number of principal components to keep.
    
    Returns:
    X_reduced : numpy.ndarray
        The data projected into the lower-dimensional space.
    """
    
    # Step 1: Mean Center the data
    X_mean = np.mean(X, axis=0)
    centered_data = X - X_mean

    # Step 2: Calculate Covariance matrix
    cov_matrix = np.cov(centered_data.T)

    # Step 3: Compute eigenvalues and eigenvectors
    eigen_vals, eigen_vecs = np.linalg.eig(cov_matrix)

    # Step 4: Sort eigenvalues and corresponding eigenvectors in decreasing order
    idx_sorted_decreasing = np.argsort(eigen_vals)[::-1]
    eigen_vals_sorted = eigen_vals[idx_sorted_decreasing]
    angle = eigen_vals_sorted[:,]
    eigen_vecs_sorted = eigen_vecs[:, idx_sorted_decreasing]  # Corrected: Sort column-wise

    # Step 5: Select the top n_components eigenvectors
    eigen_vecs_subset = eigen_vecs_sorted[:, :n_components]

    # Step 6: Transform the data (project onto the new subspace)
    X_reduced = np.dot(centered_data, eigen_vecs_subset)

    return X_reduced


In [37]:
X_sample = np.random.rand(100, 5)  # 100 samples, 5 features
X_reduced = compute_pca(X_sample, n_components=2)
print(X_reduced)

[[ 0.30127926 -0.07153665]
 [ 0.02356571  0.10795368]
 [-0.46979427 -0.29834505]
 [-0.5365572   0.26181762]
 [-0.07179863  0.58271646]
 [ 0.16174147  0.45345974]
 [-0.10338027  0.15144529]
 [-0.00308307 -0.23848444]
 [ 0.18731793 -0.1860031 ]
 [-0.08660801  0.1950965 ]
 [-0.28353898 -0.34127674]
 [ 0.12252711 -0.63472579]
 [-0.29860216  0.24181588]
 [-0.22344147  0.33880445]
 [ 0.28164284 -0.08917147]
 [ 0.71590282 -0.47006415]
 [ 0.15256182 -0.17955227]
 [-0.39516275 -0.52608758]
 [ 0.63920177  0.68245941]
 [-0.18177197 -0.08518042]
 [-0.22862541  0.12132878]
 [ 0.021786    0.07261526]
 [-0.04334624  0.39070019]
 [ 0.03698006  0.06548567]
 [-0.43569335  0.36210103]
 [ 0.48105184 -0.44098804]
 [-0.08122268 -0.07021245]
 [ 0.49848873  0.23528403]
 [-0.29827854  0.12008631]
 [-0.29185762  0.18139257]
 [ 0.04318283 -0.07146148]
 [ 0.46771932 -0.31894354]
 [ 0.69034537 -0.15355969]
 [ 0.58746729 -0.18832098]
 [-0.12961854 -0.62156883]
 [ 0.2492036  -0.22562629]
 [-0.59119221  0.44498042]
 